### **Classification and Sorting of Coffee beans Using Deep Learning.**

# **DATA**

## Read and access data from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Import the required packages

In [ ]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.utils.data as data
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time, os, copy, argparse
import multiprocessing
from torchsummary import summary
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
!pip install tensorboard

## Launch the program code with the environment variable CUDA_LAUNCH_BLOCKING

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

## Create train, validation and test from directory paths in Google Drive

In [ ]:
# Create train, validate and test from the dataset path
train_dir = '/content/drive/MyDrive/Coffee/dataset/train'

val_dir = '/content/drive/MyDrive/Coffee/dataset/val'

test_dir = '/content/drive/MyDrive/Coffee/dataset/test'

## Create a starting variable for the experiment

In [ ]:
# Create variables for multiple epochs
num_epochs = 25

# Create variables for batch size
batch_size = 4

# variable for the number of classes
num_classes = 4

# variable for num_workers which tells how many processor used to decomposethe data
num_cpu = multiprocessing.cpu_count()

## Calculate mean and standard deviation (STD) values for image normalization

In [ ]:
# load and display an example of the image of the coffee bean that will be calculate

# Loading images
path_base ='/content/drive/MyDrive/Coffee/dataset/train'
img_path = path_base + '/longberry/1010.jpg'
img = Image.open(img_path)
img

In [ ]:
# Convert PIL image to numpy array
img_np = np.array(img)


# Plots the value of each pixel of the image
plt.hist(img_np.ravel(), bins=50, density=True)
plt.xlabel("Pixel Value")
plt.ylabel("frequency")
plt.title("Distribution of each pixel")

In [ ]:
# Create image tranforms to PyTorch tensor
transform = transforms. Compose([
    transforms. ToTensor()
])

# Transform from PIL image to tensor
img_tr = transform(img)

# Convert tensor images to numpy arrays
img_np = np.array(img_tr)

# Plots tha value of each pixel of the image
plt.hist(img_np.ravel(), bins=50, density=True)
plt.xlabel("Pixel Value")
plt.ylabel("frequency")
plt.title("Distribution of each pixel")

In [ ]:
# Calculate mean and standard deviasi (STD) values

# Tensor image
img_tr = transform(img)

# Calculating mean and std
mean, std = img_tr.mean([1,2]), img_tr.std([1,2])

# Print the calculation results
print("Average value (mean) and standard deviation (STD) before normalization:")
print("Average (mean) of the image:", mean)
print("Standard deviation (std) of the image:", std)

In [ ]:
# Compare the value before normalization with thenormalized value

# Create image tranforms to PyTorch tensor and normalization of mean and std values
transform_norm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Of images to be normalized
img_normalized = transform_norm(img)

# Convert normalized image to numpy arrays
img_np = np.array(img_normalized)

# Plots the value of each pixel of the image
plt.hist(img_np.ravel(), bins=50, density=True)
plt.xlabel("Pixel value")
plt.ylabel("frequency")
plt.title("Distribution of each pixel")

In [ ]:
# Displaying normalized image results

img_normalized = transform_norm(img)

# Convert images to numpy arrays
img_normalized = np.array(img_normalized)

# Transpose from shape(3,,) to shape(,,3)
img_normalized = img_normalized.transpose(1, 2, 0)

# Displaying the results of normalized images
plt.imshow(img_normalized)
plt.xticks([])
plt.yticks([])

In [ ]:
# Calculate the mean and std values after normalization where the mean value is 0.0 and the std value of normalized images

img_nor = transform_norm(img)

# Calculate the average value and standard deviation
mean, std = img_nor.mean([1,2]), img_nor.std([1,2])

# Prints the results of the average score and standard of the deviasi
print("Average values and standard deviations from the normalization of the image:")
print("Average value of the image:", mean)
print("Standard value deviation from the image:", std)

## Creating Data Transform

In [ ]:
# Create the mean value variables (mean) and standard deviation (STD) for use in the normalization of the image
mean = [0.7980, 0.7820, 0.7562]
std = [0.1441, 0.1729, 0.2341]

In [ ]:
# Transform image data to match when training
image_transforms = {
    'train': transforms.Compose([
                                 transforms.Resize(size=(256, 256)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean, std)
                                 ]),
    'val': transforms.Compose([
                               transforms.Resize(size=(256, 256)),
                               transforms.ToTensor(),
                               transforms.Normalize(mean, std)
                               ]),
    'test': transforms.Compose([
                                transforms.Resize(size=(256, 256)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)
                                ]),
                    }

## Dataset Summary and Visualization

In [ ]:
# Load data from a folder
dataset = {
    'train': datasets.ImageFolder(root=train_dir, transform=image_transforms['train']),
    'val': datasets.ImageFolder(root=val_dir, transform=image_transforms['val']),
    'test': datasets.ImageFolder(root=test_dir, transform=image_transforms['test']),
}

In [ ]:
# Load Multiple classes on the dataset
classes = dataset['train'].classes
print('Many classes in this dataset are', len(classes),'and classes in the dataset consist of:', classes)

NameError: name 'dataset' is not defined

In [ ]:
# Calculate a lot of data from training data, validation data and test data
dataset_sizes = {
    'train':len(dataset['train']),
    'val':len(dataset['val']),
    'test':len(dataset['test'])
}

In [ ]:
# Print a large number of datasets
print("Image data in the train data:",dataset_sizes['train'])
print("Image data in the valid data:", dataset_sizes['val'])
print("Image data in the test data:", dataset_sizes['test'])

In [ ]:
# Create an iterator to load the dataset
dataloaders = {
    'train' : data.DataLoader(dataset['train'], batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True, drop_last=True),
    'val'   : data.DataLoader(dataset['val'], batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True, drop_last=True),
    'test'  : data.DataLoader(dataset['test'], batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True, drop_last=True)
}

print('a lot of the amount of training data per class is', len(dataloaders['train']), 'image')
print('a lot of validation data per class is', len(dataloaders['val']), 'image')
print('a lot of the amount of test data per class is', len(dataloaders['test']), 'image')

In [ ]:
# Check the number of batch sizes, color outputs and the number of image in pixels

images, labels = next(iter(dataloaders['train']))
print("jumlah kelas adalah", len(labels))
print("dimensi ukuran gambar adalah", images.shape)

In [ ]:
# visualization of image data on datasets

def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = [0.7980, 0.7820, 0.7562]
    std = [0.1441, 0.1729, 0.2341]
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # pause to update the plot of the image

# Retrieving image data from training data
inputs, classes = next(iter(dataloaders['train']))
class_names = dataset['train'].classes
# Create a grid
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

# Experiment 1

## Device default

In [ ]:
# Create the default device as GPU, if any
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
images, labels = images.to(device), labels.to(device)

## Experiment1:Resnet18

In [ ]:
# Experiment 1: Using a pretrained model with the Resnet18 architecture

print("\Load Resnet18\n")
model_ft = models.resnet18(pretrained=True)

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,num_classes )

# Transfer model to GPU
model_ft = model_ft.to(device)

for num, (name, param) in enumerate(model_ft.named_parameters()):
    print(num, name, param.requires_grad )

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Learning rate
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
# print model summary
print('Model Summary:-\n')
summary(model_ft, input_size=(3, 256, 256))
print(model_ft)

In [ ]:
writer=SummaryWriter('/content/content/runs/summary_1')
writer.add_graph(model_ft, images)

In [ ]:
# Train the model
print("\nTraining:-\n")

def train_model(model, criterion, optimizer, scheduler, num_epochs=30):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Tensorboard summary
    writer = SummaryWriter("content/runs/model_percobaan_1")

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Record training loss and accuracy for each phase
            if phase == 'train':
                writer.add_scalar('Train/Loss', epoch_loss, epoch)
                writer.add_scalar('Train/Accuracy', epoch_acc, epoch)
                writer.flush()
            else:
                writer.add_scalar('Valid/Loss', epoch_loss, epoch)
                writer.add_scalar('Valid/Accuracy', epoch_acc, epoch)
                writer.flush()

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

    writer.close()

    %load_ext tensorboard
    %tensorboard --logdir content

In [ ]:
# Training model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=num_epochs)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir content/runs
%reload_ext tensorboard

In [ ]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

## Saving the model

In [ ]:
# save model
PATH = 'model_percobaan_1.pth'
print("\nSaving the model...")
torch.save(model_ft, PATH)

## Predicting from validation data

In [ ]:
# Visualizing the model predictions

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)
        visualize_model(model_ft)

## Predict from test data

In [ ]:
# Visualizing the model predictions

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

visualize_model(model_ft)

## Load accuracy from test data

In [ ]:
def computeTestSetAccuracy(model, criterion):
    '''
    Function to compute the accuracy on the test set
    Parameters
        :param model: Model to test
        :param loss_criterion: Loss Criterion to minimize
    '''

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    test_acc = 0.0
    test_loss = 0.0

    # Validation - No gradient tracking needed
    with torch.no_grad():

        # Set to evaluation mode
        model.eval()

        # Validation loop
        for j, (inputs, labels) in enumerate((dataloaders['test'])):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, labels)

            # Compute the total loss for the batch and add it to valid_loss
            test_loss += loss.item() * inputs.size(0)

            # Calculate validation accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            # Compute total accuracy in the whole batch and add to valid_acc
            test_acc += acc.item() * inputs.size(0)

            print("Test Batch number: {:03d}, Test: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))

    # Find average test loss and test accuracy
    avg_test_loss = test_loss/(dataset_sizes['test'])
    avg_test_acc = test_acc/(dataset_sizes['test'])

    print("\nTest accuracy : " + str(avg_test_acc))

In [ ]:
computeTestSetAccuracy(model_ft, criterion)

## Perform detection tests on test data

In [ ]:
# Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in dataset['train'].class_to_idx.items()}
print(idx_to_class)

In [ ]:
def predict(model, test_image_name):
    '''
    Function to predict the class of a single test image
    Parameters
        :param model: Model to test
        :param test_image_name: Test image
    '''

    transform = image_transforms['test']


    test_image = Image.open(test_image_name)
    plt.imshow(test_image)

    test_image_tensor = transform(test_image)

    if torch.cuda.is_available():
        test_image_tensor = test_image_tensor.view(1, 3, 256, 256).cuda()
    else:
        test_image_tensor = test_image_tensor.view(1, 3, 256, 256)

#    idx_to_class = {
#      "1":"defect",
#      "2":"longberry",
#      "3":"peaberry",
#      "4":"premium",
#    }

    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(test_image_tensor)
        ps = torch.exp(out)

        topk, topclass = ps.topk(3, dim=1)
        cls = idx_to_class[topclass.cpu().numpy()[0][0]]
        score = topk.cpu().numpy()[0][0]

        for i in range(3):
            print("Prediction", i+1, ":", idx_to_class[topclass.cpu().numpy()[0][i]], ", Score: ", topk.cpu().numpy()[0][i])

In [ ]:
import torch.nn.functional as F

def predict(model, test_image_name):
    '''
    Function to predict the class of a single test image
    Parameters
        :param model: Model to test
        :param test_image_name: Test image
    '''

    transform = image_transforms['test']


    test_image = Image.open(test_image_name)
    plt.imshow(test_image)

    test_image_tensor = transform(test_image)

    if torch.cuda.is_available():
        test_image_tensor = test_image_tensor.view(1, 3, 256, 256).cuda()
    else:
        test_image_tensor = test_image_tensor.view(1, 3, 256, 256)

#    idx_to_class = {
#      "1":"defect",
#      "2":"longberry",
#      "3":"peaberry",
#      "4":"premium",
#    }

    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(test_image_tensor)
        ps = F.softmax(out.data,dim=1)

        topk, topclass = ps.topk(4, dim=1)
        cls = idx_to_class[topclass.cpu().numpy()[0][0]]
        score = topk.cpu().numpy()[0][0]

        for i in range(4):
            print("Prediction", i+1, ":", idx_to_class[topclass.cpu().numpy()[0][i]], ",", end=" ")
            print("Score: {:.4f}%".format(topk.cpu().numpy()[0][i] * 100))

## Evaluating the model

In [ ]:
nb_classes = 4

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['test']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):

    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
plot_confusion_matrix(cm           = np.array([[349.,  15.,  11.,  25.],
        [  3., 364.,   4.,  29.],
        [  6.,   0., 285., 109.],
        [ 20.,   5.,   4., 371.]]),
                      normalize    = True,
                      target_names = ['defect', 'longberry', 'peaberry', 'premium'],
                      title        = "Confusion Matrix")

In [ ]:
#To get the per-class accuracy:

print(confusion_matrix.diag()/confusion_matrix.sum(1))

# EXPERIMENT 2

## Device default

In [ ]:
# Create the default device as GPU, if any
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
images, labels = images.to(device), labels.to(device)

## Experiment 2: MobuleNetV2

In [ ]:
# Load pre-trained model as MobileNetV2
print("\nLoading mobilenetv2 as feature extractor ...\n")
model_tl = models.mobilenet_v2(pretrained=True)

# Freeze all the required layers (i.e except last conv block and fc layers)
for params in list(model_tl.parameters())[0:-5]:
  params.requires_grad = False

# Modify fc layers to match num_classes
num_ftrs=model_tl.classifier[-1].in_features

model_tl.classifier=nn.Sequential(
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(in_features=num_ftrs, out_features=num_classes, bias=True)
    )

# Transfer the model to GPU
model_tl = model_tl.to(device)

# Print model summary
print('Model Summary:-\n')
for num, (name, param) in enumerate(model_tl.named_parameters()):
    print(num, name, param.requires_grad )

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer_tl = optim.Adam(model_tl.parameters(), lr=0.001)

# Learning rate decay
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_tl, step_size=7, gamma=0.1)

In [ ]:
# print model summary
print('Model Summary:-\n')
summary(model_tl, input_size=(3, 256, 256))
print(model_tl)

In [ ]:
writer=SummaryWriter('/content/content/runs/summary_model_2')
writer.add_graph(model_tl, images)

## Train the model on experiment 2

In [ ]:
# Train the model
print("\nTraining:-\n")

def train_model(model, criterion, optimizer, scheduler, num_epochs=30):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Tensorboard summary
    writer = SummaryWriter("content/runs/model_percobaan_2")

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Record training loss and accuracy for each phase
            if phase == 'train':
                writer.add_scalar('Train/Loss', epoch_loss, epoch)
                writer.add_scalar('Train/Accuracy', epoch_acc, epoch)
                writer.flush()
            else:
                writer.add_scalar('Valid/Loss', epoch_loss, epoch)
                writer.add_scalar('Valid/Accuracy', epoch_acc, epoch)
                writer.flush()

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

    writer.close()

In [ ]:
# Training model
model_tl = train_model(model_tl, criterion, optimizer_tl, exp_lr_scheduler,
                       num_epochs=num_epochs)

In [ ]:
# Save model
PATH = 'model_percobaan_2.pth'
print("\nSaving the model...")
torch.save(model_tl, PATH)

## Predict from validation data

In [ ]:
# Visualizing the model predictions

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(model_tl)

## Predict from test data

In [ ]:
# Visualizing the model predictions

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

visualize_model(model_tl)

## Load accuracy from test data

In [ ]:
def computeTestSetAccuracy(model, criterion):
    '''
    Function to compute the accuracy on the test set
    Parameters
        :param model: Model to test
        :param loss_criterion: Loss Criterion to minimize
    '''

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    test_acc = 0.0
    test_loss = 0.0

    # Validation - No gradient tracking needed
    with torch.no_grad():

        # Set to evaluation mode
        model.eval()

        # Validation loop
        for j, (inputs, labels) in enumerate((dataloaders['test'])):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, labels)

            # Compute the total loss for the batch and add it to valid_loss
            test_loss += loss.item() * inputs.size(0)

            # Calculate validation accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            # Compute total accuracy in the whole batch and add to valid_acc
            test_acc += acc.item() * inputs.size(0)

            print("Test Batch number: {:03d}, Test: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))

    # Find average test loss and test accuracy
    avg_test_loss = test_loss/(dataset_sizes['test'])
    avg_test_acc = test_acc/(dataset_sizes['test'])

    print("\nTest accuracy : " + str(avg_test_acc))

In [ ]:
computeTestSetAccuracy(model_tl, criterion)

## Perform detection tests on test data

## Evaluatin the Model

In [ ]:
nb_classes = 4

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['test']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_tl(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
plot_confusion_matrix(cm           = np.array([[297.,  39.,  36.,  28.],
                                               [  8., 364.,  12.,  16.],
                                               [ 14.,   7., 311.,  68.],
                                               [ 46.,  25.,  25., 304.]]),
                      normalize    = True,
                      target_names = ['defect', 'longberry', 'peaberry', 'premium'],
                      title        = "Confusion Matrix")

In [ ]:
#To get the per-class accuracy:

print(confusion_matrix.diag()/confusion_matrix.sum(1))